## Setup for Google Colab

In [ ]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [ ]:
%%shell
chmod +x organize_food101.sh
./organize_food101.sh

In [10]:
from __future__ import print_function
import os
import torch
import torch.nn as nn
from lr_model import LRModel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import itertools
import utils
%matplotlib inline

In [ ]:
dataset_name = 'folder'
dataroot = 'food-101'
workers = 2
batchSize = 64
imageSize = 64
nz = 100
ngf = 64
ndf = 64
niter = 25
lr = 0.0002
beta1 = 0.5
ngpu = 1
netGPath = ''
netDPath = ''
outf = ''
torch.manual_seed(1)

## Load the food dataset 

In [ ]:
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

dataroot = 'food-101/images'

data_transforms = {
    'train': transforms.Compose([
                                   transforms.Resize(imageSize),
                                   transforms.CenterCrop(imageSize),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ]),
    'test': transforms.Compose([
                                   transforms.Resize(imageSize),
                                   transforms.CenterCrop(imageSize),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ]),
}

image_datasets = {x: dset.ImageFolder(os.path.join(dataroot, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}

In [ ]:
def svm_loss(scores, targets, margin=1):
    """Computes SVM loss.
    
    Args:
        scores: `torch.tensor` of shape (num_examples, dimension)
        targets: `torch.tensor` of size (num_examples)
    
    Returns: Cross SVM Loss
    """
    N = scores.shape[0]
    correct_scores = scores[range(N), targets].view(-1, 1)
    margins = torch.clamp(scores - correct_scores + margin, min=0.0)
    margins[range(N), targets] = 0
    loss = torch.sum(margins) / N
    
    return loss

In [1]:
def train(model, dataloader,model_svm, lr=0.001, momentum=0.9, batch_size=64, epochs=10, reg_strength=0.003):

    # create optimizer
    optimizer = optim.SGD(model_svm.parameters(), lr=lr, momentum=momentum)

    num_iters = 0
    losses = []
    correct = 0
    total = 0
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(dataloader, 0):
            # zero gradients
            optimizer.zero_grad()

            images = data[0].to(device)
            labels = data[1].to(device)


            svm_inputs = model.extract_features(images)

            svm_out = model_svm(svm_inputs)


            # forward, backward and optimize
            outputs = model_svm(svm_inputs)
            loss = svm_loss(outputs, labels)
            
            # add regularization, only for weights not for biases
            regW = 0
            for p in model.named_parameters():
                if p[0].endswith('weight'):
                    regW += torch.sum(p[1]**2)
            loss += reg_strength * regW
            
            
            loss.backward()
            optimizer.step()
            
            _, predicted = torch.max(outputs.data, 1)

            # Total number of labels
            total += labels.size(0)

            # Total correct predictions
            correct += (predicted.cpu() == labels.cpu()).sum()

            
            num_iters += 1
            losses.append(loss.item())

            # print statistics every 1000th iteration
            if num_iters % 1000 == 0:       
                accuracy = 100 * correct.double() / total
                print('Iteration: {0}, Epoch: {1}, loss: {2:.3f}, Accuracy: {3:.2f}'.format(num_iters, epoch, loss.item(), accuracy.item()))
                
    
    # losses per iteration
    return losses

In [ ]:
netD = utils.getDiscriminatorModel(netDpath='netD_cifar.pth')

In [ ]:
model_svm = LRModel(15360, 101, 5000)
model_svm.to(device)
losses = train(model=netD2, dataloader=dataloaders['train'],model_svm=model_svm, epochs=10)